In [ ]:
import json, os, random, re, requests, unicodedata, urllib, sys

In [ ]:
# TODO: Getting the refresh_token should happen online with an OAuth dialog, and the refresh token
# and client secret should be stored here instead of the username and password.
# Or we should have a onetime username/password dialog in Python, and store same.

# ~/.config/esdr/auth.json should look like:
# {
#    "grant_type" : "password",
#    "client_id" : <client id>,
#    "client_secret" : <client secret>,
#    "username" : <email address>,
#    "password" : <actual password>
# }

class Esdr:
    def __init__(self, auth_file, prefix='https://esdr.cmucreatelab.org', user_agent='esdr-library.py'):
        self.prefix = prefix
        self.auth_file = auth_file
        self.tokens = None
        self.user_agent = user_agent
        self.product_cache = {}
        self.feed_cache = {}

    @staticmethod
    def save_client(destination, display_name, username='EDIT ME', password='EDIT ME'):
        client_secret = '%032x' % random.randrange(16**32)
        client_name = re.sub('\W+', '_', display_name)

        data = {'displayName' : display_name,
                'clientName' :  client_name,
                'clientSecret' : client_secret}

        auth = {
            'grant_type': 'password',
            'client_id': data['clientName'],
            'client_secret': client_secret,
            'username': username,
            'password': password
        }
        
        open(destination, 'w').write(json.dumps(auth))
        print('Wrote %s to %s' % (json.dumps(auth), destination))
        
        print('Please create a client here: https://esdr.cmucreatelab.org/home/clients')
        print('with these parameters:')
        print('Display Name: %s' % display_name)
        print('Client ID: %s' % client_name)
        print('Client Secret: %s' % client_secret)
        
        if username == 'EDIT ME' or password == 'EDIT ME':
            print()
            print('and edit usernamd and password in %s' % destination)
        
        print()
        print('Instantiate ESDR client like so:')
        print("esdr = Esdr('%s')" % destination)
    
    @staticmethod
    # Replace sequences of non-word characters with '_'
    def make_identifier(name):
         name = ''.join(c for c in unicodedata.normalize('NFD', unicode(name))
                      if unicodedata.category(c) != 'Mn')
         return re.sub(r'\W+', '_', name).strip('_')

    def api(self, http_type, path, json_data=None, oauth=True):
        kwargs = {
            'headers': {
                'User-Agent': self.user_agent
            },
            'timeout': 4 * 60
        }
        
        if json_data:
            if http_type == 'GET':
                # get params
                kwargs['params'] = json_data
            else:
                # post JSON directly
                kwargs['json'] = json_data

        if oauth:
            kwargs['headers']['Authorization'] = 'Bearer %s' % self.get_access_token()

        url = self.prefix + path

        fn = {'GET':requests.get, 'POST':requests.post, 'PUT':requests.put, 'DELETE':requests.delete, 'PATCH':requests.patch}[http_type]
        
        maxRetries = 5
        for attempt in range(1, maxRetries+1):
            try:
                r = fn(url, **kwargs)
                if attempt > 1:
                    print('ESDR.api: Attempt %d succeeded' % attempt)
                break
            except requests.Timeout:
                print('ESDR.api: Timeout during attempt %d.' % attempt)
                if attempt == maxRetries:
                    print('ESDR.api: No more retries, raising exception')
                    raise
                else:
                    print('ESDR.api: Retrying.')
        
        r.raise_for_status()
        return r.json()

    def get_access_token(self):
        if not self.tokens:
            self.get_tokens()
        return self.tokens['access_token']
    
    def get_tokens(self):
        try:
            auth = json.load(open(self.auth_file))
        except Exception as e:
            raise Exception('While trying to read authorization file %s, %s' % (self.auth_file, e), sys.exc_info()[2])
        self.tokens = self.api('POST',
                               '/oauth/token',
                               json.load(open(self.auth_file)),
                               oauth=False)

    def query(self, path, args):
        response = self.api('GET', path, args)
        return response['data']['rows']
    
    def query_first(self, path, args):
        rows = self.query(path, args)
        if len(rows) == 0:
            return None
        else:
            return rows[0];

    def get_or_create_product(self, prettyName, vendor=None, description=None, default_channel_specs={}):
        name = re.sub('\W+', '_', prettyName)
        if not vendor:
            vendor = name
        if not description:
            description = prettyName
        product = self.get_product_by_name(name)
        if not product:
            self.create_product(name, prettyName, vendor, description, default_channel_specs)
            product = self.get_product_by_name(name)
        return product
        
    def create_product(self, name, pretty_name, vendor, description, default_channel_specs={}):
        return self.api('POST', '/api/v1/products', {
            'name': name,
            'prettyName': pretty_name,
            'vendor': vendor,
            'description': description,
            'defaultChannelSpecs': default_channel_specs
        })
    
    def get_product_by_name(self, name):
        return self.query_first('/api/v1/products', {'where':'name=%s' % name})

    def get_product_by_id(self, id):
        return self.query_first('/api/v1/products', {'where':'id=%d' % id})

    def get_feed_by_id(self, id):
        return self.query_first('/api/v1/feeds', {'where':'id=%d' % id})

    def get_or_create_device(self, product, serial_number, name=None):
        device = self.get_device_by_serial_number(product, serial_number)
        if not device:
            self.create_device(product, serial_number, name=name)
            device = self.get_device_by_serial_number(product, serial_number)
        return device
    
    def get_device_by_serial_number(self, product, serial_number):
        response = self.api('GET', '/api/v1/devices', {'whereAnd': 'productId=%d,serialNumber=%s' % (product['id'], serial_number)})
        if response['data']['totalCount'] == 0:
            return None
        elif response['data']['totalCount'] == 1:
            return response['data']['rows'][0]
        else:
            raise Exception('get_device_by_serial_number: found more than one device?')

    def create_device(self, product, serial_number, name=None):
        if name == None:
            name = serial_number
        print('Creating device serialNumber %s, name %s' % (serial_number, name))
        device = self.api('POST',
                          '/api/v1/products/%d/devices' % product['id'], 
                          {
                              'name':name,
                              'serialNumber':serial_number
                          })['data']
        return device

    def get_or_create_feed(self, device, lat=None, lon=None):
        feed = self.get_feed(device, lat=lat, lon=lon)
        if not feed:
            self.create_feed(device, lat=lat, lon=lon)
            feed = self.get_feed(device)
        return feed

    # Returns the most-recently created feed belonging to the given device with the specified lat/lon OR,
    # if no such feed exists, creates a feed for the device with the given name, lat, and long.
    def get_or_create_feed_with_name(self, device, feed_name, lat=None, lon=None, exposure="outdoor"):
        feed = self.get_feed(device, lat=lat, lon=lon, order_by='-id')
        if not feed:
            response = self.create_feed_with_name(device, feed_name, lat=lat, lon=lon, exposure=exposure)
            if response:
                if response['code'] == 201:
                    feed_id = response['data']['id']
                    feed = self.get_feed_by_id(feed_id)
                else:
                    raise Exception(f"get_or_create_feed_with_name: failed to create feed, HTTP {response['code']}")
            else:
                raise Exception('get_or_create_feed_with_name: failed to create feed, no response')
        return feed

    def get_feed(self, device=None, device_id=None, lat=None, lon=None, order_by=None):
        if not device_id:
            if device:
                device_id = device['id']
            else:
                raise Exception('Must pass device or device_id to get_feed')
        query_params = {'where':'deviceId=%d' % device_id}
        if order_by:
            query_params['orderBy'] = order_by
        rows = self.query('/api/v1/feeds', query_params)
        #if a device has been moved and thus has multiple feeds, return the feed corresponding to the passed-in location
        if lat and lon:
            for row in rows:
                # Check epsilon of .000004 before a 6th decimal place would cause a round to 5 places to change value.
                latDiff = abs(row['latitude'] - lat) <= 6e-6
                lonDiff = abs(row['longitude'] - lon) <= 6e-6
                if latDiff and lonDiff:
                    return row
            return None 
        #null case
        elif len(rows) == 0:
            return None
        #if a device has multiple feeds but no location was passed in,
        #or a device has only one feed, return the first
        return rows[0]

    def create_feed(self, device, lat=None, lon=None):
        product = self.get_product_by_id(device['productId'])
        name = (device['name'] + ' ' + product['name'])
        fields = {
                    'name': name,
                    'exposure':'outdoor',
                    'isPublic':1,
                    'isMobile':0
                 }
        if lat != None:
            fields['latitude'] = lat
        if lon != None:
            fields['longitude'] = lon
        print('Creating feed %s' % fields)
        response = self.api('POST', '/api/v1/devices/%d/feeds' % device['id'], fields)
        return response

    def create_feed_with_name(self, device, feed_name, lat=None, lon=None, exposure='outdoor'):
        product = self.get_product_by_id(device['productId'])

        fields = {
                    'name': feed_name,
                    'exposure':exposure,
                    'isPublic':1,
                    'isMobile':0
                 }
        if lat != None:
            fields['latitude'] = lat
        if lon != None:
            fields['longitude'] = lon
        print('Creating feed %s' % fields)
        response = self.api('POST', '/api/v1/devices/%d/feeds' % device['id'], fields)
        return response

    def update_feed_name(self, feed_id, new_name):
        if not feed_id:
            return "No feed ID provided"

        if not new_name:
            return "No feed name provided"

        json = { "op" : "replace", "path" : "/name", "value" : new_name }
        response = self.api('PATCH', '/api/v1/feeds/%d/' % feed_id, [json])
        return response
    
    def update_feed_exposure(self, feed_id, new_exposure):
        if not feed_id:
            return "No feed ID provided"

        if new_exposure not in ['indoor', 'outdoor']:
            return "exposure field not of valid type ['indoor', 'outdoor']"

        json = { "op" : "replace", "path" : "/exposure", "value" : new_exposure }
        response = self.api('PATCH', '/api/v1/feeds/%d/' % feed_id, [json])
        return response

    def delete_feed_by_id(self, feed_id):
        if not feed_id:
            return "No feed ID provided"
            
        response = self.api('DELETE', '/api/v1/feeds/%d/' % feed_id)
        return response
    
    # data is of form
    # {'channel_names': ['a', 'b', 'c'],
    #  'data': [[1417524480, 1, 1, 1],
    #           [1417524481, 2, 3, 4],
    #           [1417524482, 3, 5, None]]}
    # None translates into NULL, which is a no-op
    # False translates into false, which will delete a sample already present at that time

    def upload(self, feed, data):
        return self.api('PUT', '/api/v1/feeds/%s' % feed['id'], data)

    # data = [{time:epoch1, x:3, y:4, ...},
    #         {time:epoch2, x:4, y:5, ...},
    #         ...
    #         {time:epochN, x:8, y:9, ...}]
    #
    # Not all columns need to have identical keys;  data will be uploaded with union of all column names
    # and gaps will be sent as NULL
    #
    # Requires "import pandas as pd"

    def upload_dicts(self, feed, data):
        df = pd.DataFrame(data)

        # Replace NaN with None
        df = df.where(pd.notnull(df), None)

        # Move time to beginning since that's what ESDR expects
        if not 'time' in set(df.columns):
            raise Exception('"time" must be a column for upload_dicts, but only %s provided' % list(df.columns))
        keys = ['time'] + sorted(set(df.columns)-set(['time']))
        df = df[keys]

        esdrValues = df.values.tolist()

        # TIME is implicit for ESDR;  don't list in channel_names
        esdr_upload = {'channel_names':keys[1:], 'data':esdrValues}

        esdr.upload(feed, esdr_upload)
        print('uploaded %d records of %d values to %s/%s/%s' % (len(esdrValues), len(esdrValues[0]), feed['name'], feed['deviceId'], feed['id']))

    def get_tile_prefix(self, feed, channel_name):
        return self.prefix + '/api/v1/feeds/%d/channels/%s/tiles' % (feed['id'], channel_name)

    def get_or_create_feed_from_device_info(self, device_info):
        product = self.get_or_create_product(device_info['product'])

        device = self.get_device_by_serial_number(product, device_info['serialNumber'])
        if not device:
            self.create_device(product, device_info['serialNumber'])
            device = esdr.get_device_by_serial_number(product, device_info['serialNumber'])

        feed = esdr.get_feed(device)
        if not feed:
            lat = None
            lon = None
            if 'lat' in device_info:
                lat = device_info['lat']
                lon = device_info['lon']
            esdr.create_feed(device, lat=lat, lon=lon)
            feed = esdr.get_feed(device)
        return feed

    def cached_get_or_create_product(self, product_name):
        if not product_name in self.product_cache:
            self.product_cache[product_name] = self.get_or_create_product(product_name)
        return self.product_cache[product_name]

    def cached_get_or_create_product_device_feed(
        self, product_name, device_serial_number, lat=None, lon=None):

        product = self.cached_get_or_create_product(product_name)

        cache_key = (product['id'], device_serial_number)

        if not (cache_key in self.feed_cache):
            device = self.get_or_create_device(product, device_serial_number)
            feed = self.get_or_create_feed(device, lat=lat, lon=lon)
            self.feed_cache[cache_key] = feed

        return self.feed_cache[cache_key]

#Create product:
#esdr.api('POST', '/api/v1/products', {
#   "name" : "ACHD",
#   "prettyName" : "ACHD",
#   "vendor" : "ACHD",
#   "description" : "A sensor operated by the Allegheny County Health Department (ACHD)",
#   "defaultChannelSpecs" : {}
#})


In [ ]:
#esdr = Esdr()
#product = esdr.get_product_by_name('TestProduct')
#device = esdr.get_or_create_device(product, 'TestDevice')
#feed = esdr.get_or_create_feed(device)
#data = {'channel_names': ['a', 'b', 'c'],
#        'data': [[1417524480, 1, 1, 1],
#                 [1417524481, 2, 3, 4],
#                 [1417524482, 3, 5, 7]]}
#
#esdr.upload(feed, data)

In [ ]:
# Esdr.create_client('timemachine1 airnow uploader')
